# 张量
## 张量类型：
- torch.FloatTensor 32位浮点型
- torch.DoubleTensor 64位浮点型
- torch.ShortTensor 16位整形
- torch.IntTensor 32位整形
- torch.LongTensor 64位整形
## 创建常量

### torch.Tensor()创建的是torch.FloatTensor类型

In [1]:
import torch
a  = torch.Tensor([[1, 2],[2, 3]])
a

tensor([[1., 2.],
        [2., 3.]])

In [2]:
b = torch.LongTensor([[2],[4]])
b

tensor([[2],
        [4]])

In [3]:
# 创建全零tensor
torch.zeros((2,3))

tensor([[0., 0., 0.],
        [0., 0., 0.]])

**可以像numpy一样通过索引方式获取其中的元素，也可以改变它的值**

In [4]:
a[0][0] = 100
a

tensor([[100.,   2.],
        [  2.,   3.]])

In [5]:
numpy_a = a.numpy()
numpy_a

array([[100.,   2.],
       [  2.,   3.]], dtype=float32)

**从numpy中创建tensor**

In [6]:
import numpy as np
b = np.arange(24).reshape(2, 3, 4)
print(b)
tensor_b = torch.from_numpy(b)
tensor_b

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]


tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]], dtype=torch.int32)

### tensor存放的位置CPU或者GPU

In [7]:
# 查看是否支持GPU
torch.cuda.is_available()

True

In [8]:
tensor_b.device

device(type='cpu')

In [9]:
gpu_b = tensor_b.cuda()
gpu_b.device

device(type='cuda', index=0)

# 创建变量
变量和张量本身没什么区别，只是变量会被放到计算图中，然后进行前向、反向传播，自动求导。

**变量的三个属性**
<img src='../images/001.png'>

### 当对求导的变量是标量时

In [10]:
x = torch.autograd.Variable(torch.tensor(1, dtype=torch.float), requires_grad=True)

In [11]:
w = torch.autograd.Variable(torch.tensor(2, dtype=torch.float), requires_grad=True)

In [12]:
b = torch.autograd.Variable(torch.tensor(3, dtype=torch.float), requires_grad=True)

In [13]:
y = w * x + b

In [14]:
print(x.data, x.grad, x.grad_fn)
print(w.data, w.grad, w.grad_fn)
print(b.data, b.grad, b.grad_fn)
print(y.data, y.grad, y.grad_fn)

tensor(1.) None None
tensor(2.) None None
tensor(3.) None None
tensor(5.) None <AddBackward0 object at 0x0000020403542308>


In [15]:
y.backward()

**等价于y.backward(torch.FloatTensor([1]))**

In [16]:
print(x.data, x.grad, x.grad_fn)
print(w.data, w.grad, w.grad_fn)
print(b.data, b.grad, b.grad_fn)
print(y.data, y.grad, y.grad_fn)

tensor(1.) tensor(2.) None
tensor(2.) tensor(1.) None
tensor(3.) tensor(1.) None
tensor(5.) None <AddBackward0 object at 0x0000020403547708>


### 当对求导的变量是向量时
需要在backward（）中传入参数，为相同维数的张量

In [17]:
x = torch.randn(3)
x

tensor([-0.4315, -1.7685, -0.8425])

In [18]:
x = torch.autograd.Variable(x, requires_grad=True)

In [19]:
y = x ** 2

In [20]:
print(x , x.data, x.grad, x.grad_fn)
print(y, y.data, y.grad, y.grad_fn)

tensor([-0.4315, -1.7685, -0.8425], requires_grad=True) tensor([-0.4315, -1.7685, -0.8425]) None None
tensor([0.1862, 3.1276, 0.7099], grad_fn=<PowBackward0>) tensor([0.1862, 3.1276, 0.7099]) None <PowBackward0 object at 0x000002040352D388>


In [21]:
# 会出错
y.backward()

RuntimeError: grad can be implicitly created only for scalar outputs

In [22]:
# 需要传入相同维数的向量
y.backward(torch.tensor([1, 0.1, 0.01]))

In [23]:
print(x , x.data, x.grad, x.grad_fn)
print(y, y.data, y.grad, y.grad_fn)

tensor([-0.4315, -1.7685, -0.8425], requires_grad=True) tensor([-0.4315, -1.7685, -0.8425]) tensor([-0.8630, -0.3537, -0.0169]) None
tensor([0.1862, 3.1276, 0.7099], grad_fn=<PowBackward0>) tensor([0.1862, 3.1276, 0.7099]) None <PowBackward0 object at 0x00000204038016C8>


[ -0.8630, -0.3537, -0.0169]各是原来梯度的1、 0.1、 0.01倍

In [24]:
a = torch.randn(3)
a = torch.autograd.Variable(a, requires_grad=True)
z = a ** 2
z.backward(torch.tensor([1,1,1]))
print(a , a.data, a.grad, a.grad_fn)
print(z, z.data, z.grad, z.grad_fn)

RuntimeError: Expected isFloatingType(grads[i].type().scalarType()) to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)